### Dataset
This is the paper that contains properties of ~134k organic molecules with up to 9 heavy atoms (C,N,O,F).

Quantum chemistry structures and properties of 134 kilo molecules
Raghunathan Ramakrishnan, Pavlo O. Dral, Matthias Rupp & O. Anatole von Lilienfel
Scientific Data volume 1, Article number: 140022 (2014) d
https://www.nature.com/articles/sdata201422

We will use simple GCNN to predict Gibbs free energy of the molecules.
.

### Program to install
For graph convolution, we will need to install extra package. You need to install torch_geometric and rdkit.

On Google Colab, you can use the following command for rdkit:

!pip install rdkit

For CPU,

!pip install torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cpu.html

For CUDA 11.8,

!pip install torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cu118.html

For CUDA 12.1,

!pip install torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

Please check the official pytorch geometric website for the install instruction: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html



### Data Example
In the class you should have downloaded the SmilesAndG.json. Let's open it and utilize rdkit.

In [ ]:
import json
data = json.load(open('SmilesAndG.json'))
d = data[100]
print(f'Smiles of the 100th data is {d[0]}')
print(f'This smiles has a Gibbs free energy of {d[1]} Hatree (Ha; 1 Ha = 2625.5 kJ/mol)')
print('Below shows the structure')
from rdkit import Chem
Chem.MolFromSmiles(d[0])

## Homework part (a) Define a featurization scheme
Instruction:

Below is the processing method for converting the data. Fill in the gap below to create one hot encoding features for atom and edges. The list of column attributes are written in unique_atoms and unique_bonds.

In [ ]:
import torch
unique_atoms = ['H','C','N','O','F']
unique_bonds = ['SINGLE', 'DOUBLE', 'TRIPLE', 'AROMATIC']
def ProcessDatum(d):
    mol = Chem.MolFromSmiles(d[0]) 
    mol = Chem.AddHs(mol)
    v = []
    for atom in mol.GetAtoms():
        ######################## Fill in here ########################
        oh = [0 for _ in unique_atoms]
        oh[unique_atoms.index(atom.GetSymbol())] = 1
        ######################## Fill in here ########################
        v.append(oh)
    v = torch.tensor(v,dtype=torch.float)
    c = []
    e = []
    for bond in mol.GetBonds():
        c.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
        c.append([bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()]) # torch_geometric requires bi_directional graph
        ######################## Fill in here ########################
        oh = [0 for _ in unique_bonds]
        oh[unique_bonds.index(str(bond.GetBondType()))] = 1
        ######################## Fill in here ########################
        e.append(oh)
        e.append(oh)
    c = torch.tensor(c,dtype=torch.long).T # torch wants transposed matrix
    e = torch.tensor(e,dtype=torch.float)
    y = torch.tensor([d[1]*2625.5]) # convert to kJ/mol
    return v,e,c,y

## Homework part (b)
Instruction:

Use the torch_geometric.data.Data to make a graph datum in the Fill in here. The information is in the lecture as well as the torch_geometric website (https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html#data-handling-of-graphs)

In [ ]:
import torch
from torch_geometric.data import Data, Dataset
import os
from tqdm import tqdm

class SmilesData(Dataset):
    def __init__(self, root='./'):
        super().__init__(root, transform=None, pre_transform=None, pre_filter=None)
        data = json.load(open(os.path.join(root,'SmilesAndG.json')))
        self.data = []
        for d in tqdm(data):
            v,e,c,y = ProcessDatum(d)
            ######################## Fill in here ########################
            datum = Data(x=v,edge_index=c,edge_attr=e,y_raw=y)
            ######################## Fill in here ########################
            self.data.append(datum)
        # The training is significantly better if atomic contribution is calculated and subtracted.
        x = []
        y_raw = []
        for d in self.data:
            x.append(d.x.sum(0))
            y_raw.append(d.y_raw)
        x = torch.stack(x)
        y_raw = torch.cat(y_raw)
        self.atomistic_contribution = torch.linalg.lstsq(x,y_raw).solution
        y_raw = y_raw - x@self.atomistic_contribution
        for d,yy in zip(self.data,y_raw):
            d.y_raw = yy

    def len(self):
        return len(self.data)

    def get(self, idx):
        return self.data[idx]
Dataset = SmilesData()

The following code randomly split the data into 80% train, 10% validation and 10% test. The output mean and standard deviation using the training set, and is used to apply the normalization to all data set.

In [ ]:
from torch.utils.data import random_split
data_train, data_val, data_test = random_split(Dataset,[0.80,0.1,0.1])
Y_train = []
for d in data_train:
    Y_train.append(d.y_raw)
Y_train = torch.stack(Y_train)
Ymean = Y_train.mean(0)
Ystd = Y_train.std(0)
for d in Dataset:
    d.y = (d.y_raw-Ymean)/Ystd

## Homework part (c)
Here we will use CGCNN convolution layer 

<img src="https://journals.aps.org/prl/article/10.1103/PhysRevLett.120.145301/figures/1/medium">

The atom and edge features are first "embedded" through linear layer which means that the features are converted to latent vectors. Then, the atom features will be updated according to the crystal graph convolution introduced during the class.

torch_geometric already has CGConv with input argument:
- channels (int) -  feature size of atom
- dim (int) - feature size of edge
- aggr (int) - aggregation method (add, mean, max)
- batch_norm (bool) - whether or not peform batch normalization

Instruction:

Here we already have the code to embed the atom and edge feature. Fill the blank to implement the CGCNN convonlution layer.

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import CGConv
from torch_geometric.nn.pool import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.atom_embedding = nn.Linear(5,16)
        self.edge_embedding = nn.Linear(4,16)
        ######################## Fill in here ########################
        self.conv1 = CGConv(channels=16,dim=16,aggr='mean',batch_norm=True)
        self.conv2 = CGConv(channels=16,dim=16,aggr='mean',batch_norm=True)
        ######################## Fill in here ########################
        self.out_linear = nn.Linear(16,1)
    def forward(self, batch):
        v, edge_index, e = batch.x, batch.edge_index, batch.edge_attr
        v = self.atom_embedding(v)
        e = self.edge_embedding(e)
        v = self.conv1(x=v, edge_index=edge_index,edge_attr=e)
        v = self.conv2(x=v, edge_index=edge_index,edge_attr=e)
        v = self.out_linear(v)
        y = global_mean_pool(v,batch.batch)
        return y.squeeze(1)

The code below will train your model and perform testing with test set.

In [ ]:
from torch_geometric.loader import DataLoader
dataloader_train = DataLoader(data_train, batch_size=128, shuffle=True)
dataloader_val = DataLoader(data_val, batch_size=128, shuffle=True)
dataloader_test = DataLoader(data_test, batch_size=128, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),0.01)
min_val_loss = torch.Tensor([float('Inf')])
train_loss_at_min_val_loss = torch.Tensor([float('Inf')])
min_val_loss = min_val_loss.to(device)
for i in range(100): 
    train_loss = 0
    model.train()
    for batch in dataloader_train:
        batch = batch.to(device)
        yhat= model(batch)
        optimizer.zero_grad()
        loss = criterion(yhat,batch.y)
        loss.backward()
        train_loss += loss*len(batch)
        optimizer.step()
    train_loss = train_loss/len(data_train)
    loss_val = 0
    model.eval()
    for batch in dataloader_val:
        batch = batch.to(device)
        yhat = model(batch)
        loss_val += criterion(yhat,batch.y)*len(batch)
    loss_val = loss_val/len(data_val)

    print(f'{i+1} epoch. train loss: {train_loss:.3f}, val loss: {loss_val:.3f}',end='')
    if loss_val < min_val_loss:
        torch.save(model.state_dict(),'best.pth.tar')
        min_val_loss = loss_val
        train_loss_at_min_val_loss = train_loss
        print('<-new best',end='')
    print('')
print(f'Best model: train_loss {train_loss_at_min_val_loss:.3f} val_loss {min_val_loss:.3f}')

model.load_state_dict(torch.load('best.pth.tar'))
model.eval()
YTest = []
YhatTest = []
for batch in dataloader_test:
    batch = batch.to(device)
    yhat = model(batch)
    YTest.append(batch.y)
    YhatTest.append(yhat)
YTest = torch.cat(YTest)
YhatTest = torch.cat(YhatTest)
print(f'Test loss: {criterion(yhat,batch.y):.3f}')
MeanAE = (YTest-YhatTest).abs().mean()*Ystd
print(f'MeanAE: {MeanAE:.3f} kJ/mol')